In [1]:
from fastai.vision.all import *

In [2]:
path=untar_data(URLs.PETS)/'images'

In [3]:
def is_cat(x):
  return x[0].isupper()

In [4]:
dls=ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42, label_func=is_cat, item_tfms=Resize(224)
)

In [5]:
learn=cnn_learner(dls, resnet34,  metrics=error_rate)

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 105MB/s]


In [6]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.191584,0.031119,0.010149,00:50


epoch,train_loss,valid_loss,error_rate,time
0,0.062428,0.019404,0.006766,00:49


# Image Segmentation

In [8]:
path=untar_data(URLs.CAMVID_TINY)
dls=SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames=get_image_files(path/'images'), #bs=batch-size
    label_func=lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
    codes=np.loadtxt(path/'codes.txt', dtype=str)
)

In [9]:
learn=unet_learner(dls, resnet34)
learn.fine_tune(8)

epoch,train_loss,valid_loss,time
0,2.788919,2.326476,00:01


epoch,train_loss,valid_loss,time
0,1.733471,1.422489,00:02
1,1.509338,1.235842,00:01
2,1.363031,1.046256,00:01
3,1.221599,0.809122,00:01
4,1.115616,0.795919,00:02
5,1.014479,0.686800,00:01
6,0.931121,0.665340,00:01
7,0.864811,0.653391,00:01


In [11]:
learn.show_results(max_n=6, figsize=(7, 6))

### Natural Language Processing

In [12]:
from fastai.text.all import *

In [14]:
dls=TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
learn=text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-4)

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,time
0,0.516744,0.447965,0.793240,03:41


epoch,train_loss,valid_loss,accuracy,time
0,0.471377,0.400106,0.819680,07:45
1,0.432573,0.358825,0.842840,07:44
2,0.402581,0.337428,0.854320,07:45
3,0.399909,0.334416,0.855200,07:44


In [15]:
#@ Testing:

learn.predict('I really liked that movie!!')

('pos', tensor(1), tensor([0.0201, 0.9799]))